# Update breast

In [3]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import scanpy
import scanpy as sc
from scipy.stats import spearmanr
from scvi.data import cortex, smfish
from scvi.external import GIMVI_GCN
import scvi
import pandas as pd
from sklearn.model_selection import train_test_split
import gc
import random 
random.seed(2023)


In [4]:
from scipy import stats
import scipy.stats as st
import scipy

In [5]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [6]:
# spatial_data =sc.read_h5ad("/gpfs/gibbs/pi/zhao/yl2687/data/spatial_fish_data/xenium_breast/spe_xenium_withMetrics.h5ad")
# seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")

In [ ]:
for sample_index in range(0,10):
    seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")
    spatial_data = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/human_breast_simulation/spe_xenium_data_0.1_seed{sample_index}.h5ad")
    seq_data.var_names_make_unique()
    spatial_data.var_names_make_unique()

    seq_data.obs['names'] = [i for i in range(len(seq_data))]
    spatial_data.obs['names'] = [i for i in range(len(spatial_data))]
    
    seq_data.obs['ind_x'] = [i for i in range(len(seq_data))]
    spatial_data.obs['ind_x'] = [i for i in range(len(spatial_data))]
    spatial_data.obsm['spatial'] = spatial_data.obsm['coord']

    seq_data.obs['scClassify'] = seq_data.obs['graph_cluster_anno'].copy() 

    info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

    seq_data = seq_data[:,info_gene]
    spatial_data = spatial_data[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(spatial_data .var_names).intersection(seq_data.var_names))
    g1  = sorted(g1)
    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)
    spatial_data_partial = spatial_data[:, train_g].copy()
    seq_data = seq_data.copy()

    seq_gene_names = seq_data.var_names
    n_genes = seq_data.n_vars

    # spatial_data_partial has a subset of the genes to train on
    spatial_data_partial = spatial_data_partial

    # # remove cells with no counts
    # scanpy.pp.filter_cells(spatial_data_partial, min_counts=1)
    # scanpy.pp.filter_cells(seq_data, min_counts=1)

    # setup_anndata for spatial and sequencing data
    GIMVI_GCN.setup_anndata(spatial_data_partial, obs_names = 'names')
    GIMVI_GCN.setup_anndata(seq_data) # do not register keys for scrna-seq data
    # GIMVI.setup_anndata(seq_data, labels_key="graph_cluster_anno")

    # spatial_data should use the same cells as our training data
    # cells may have been removed by scanpy.pp.filter_cells()
    spatial_data = spatial_data[spatial_data_partial.obs_names]


    scvi.settings.seed = 0
    model = GIMVI_GCN(seq_data, spatial_data_partial, n_latent = 1024)

    # train for 200 epochs
    model.train(200, batch_size = 1024)

#     _, fish_imputation_norm = model.get_imputed_values(normalized=True)
#     _, fish_imputation_raw = model.get_imputed_values(normalized=False)
    fish_imputation_norm = model.get_imputed_values(normalized=True)[0]
    fish_imputation_raw = model.get_imputed_values(normalized=False)[0]
    fish_imputation_theta = model.get_imputed_theta(normalized=False)[0]

    spatial_data_imputed = spatial_data[:, seq_data.var_names]

    spatial_data_imputed.obsm['imputed'] = fish_imputation_norm
    spatial_data_imputed.obsm['imputed_raw'] = fish_imputation_raw
    spatial_data_imputed.obsm['imputed_raw_theta'] =  fish_imputation_theta
    #     del spatial_data_imputed.uns['cell_types']

    gc.collect()
    spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_breast/gimvigcn_2020010241024_nowcheck_breast_smaple_layer2{sample_index}.h5ad")

/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cell

Epoch 200/200: 100%|██████████| 200/200 [03:42<00:00,  1.11s/it, loss=67.8, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [03:42<00:00,  1.11s/it, loss=67.8, v_num=1]


/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs

...

Epoch 200/200: 100%|██████████| 200/200 [03:47<00:00,  1.14s/it, loss=70.1, v_num=1]


/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs

Epoch 200/200: 100%|██████████| 200/200 [03:47<00:00,  1.13s/it, loss=68.5, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [03:47<00:00,  1.14s/it, loss=68.5, v_num=1]


In [8]:
seq_data

AnnData object with n_obs × n_vars = 30365 × 276
    obs: 'Barcode', 'sizeFactor', 'cluster', 'graph_cluster', 'ffpe_celltype', 'graph_cluster_anno', 'names', 'ind_x', 'scClassify', '_scvi_batch', '_scvi_labels', '_scvi_ind_x'
    var: 'ID', 'Symbol', 'Type'
    uns: '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'X_pca', 'X_umap'
    layers: 'logcounts'

# seqFish

In [1]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import scanpy
import scanpy as sc
from scipy.stats import spearmanr
from scvi.data import cortex, smfish
from scvi.external import GIMVI_GCN
import scvi
import pandas as pd
from sklearn.model_selection import train_test_split

import random 
random.seed(2023)


[rank: 0] Global seed set to 0


In [2]:
from scipy import stats
import scipy.stats as st
import scipy

In [3]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [4]:
# spatial_data =sc.read_h5ad("/gpfs/gibbs/pi/zhao/yl2687/data/spatial_fish_data/xenium_breast/spe_xenium_withMetrics.h5ad")
# seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")

In [ ]:
for sample_index in range(0,10):
    seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/seqfishdata/data/scRNAseq/seqfish/scRNAseq_seqfish.h5ad")
    spatial_data = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_seqfish/seqfish_data_seed{sample_index}.h5ad")
    seq_data.var_names_make_unique()
    
    seq_data.obs['scClassify'] = seq_data.obs['celltype'].copy() 
    spatial_data.obs['scClassify'] = spatial_data.obs['celltype_mapped_refined'].copy() 
    
    seq_data.obs['names'] = [i for i in range(len(seq_data))]
    spatial_data.obs['names'] = [i for i in range(len(spatial_data))]

    info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

    seq_data = seq_data[:,info_gene]
    spatial_data = spatial_data[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(spatial_data .var_names).intersection(seq_data.var_names))
    g1  = sorted(g1)
    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)
    spatial_data_partial = spatial_data[:, train_g].copy()
    seq_data = seq_data.copy()

    seq_gene_names = seq_data.var_names
    n_genes = seq_data.n_vars

    # spatial_data_partial has a subset of the genes to train on
    spatial_data_partial = spatial_data_partial

    # # remove cells with no counts
    # scanpy.pp.filter_cells(spatial_data_partial, min_counts=1)
    # scanpy.pp.filter_cells(seq_data, min_counts=1)

    # setup_anndata for spatial and sequencing data
    GIMVI_GCN.setup_anndata(spatial_data_partial, labels_key="scClassify", obs_names = 'names')
    GIMVI_GCN.setup_anndata(seq_data, labels_key="scClassify") # do not register keys for scrna-seq data
    # GIMVI.setup_anndata(seq_data, labels_key="graph_cluster_anno")

    # spatial_data should use the same cells as our training data
    # cells may have been removed by scanpy.pp.filter_cells()
    spatial_data = spatial_data[spatial_data_partial.obs_names]


    scvi.settings.seed = 0
    model = GIMVI_GCN(seq_data, spatial_data_partial, n_latent=32)

    # train for 200 epochs
    model.train(300)

    fish_imputation_norm = model.get_imputed_values(normalized=True)[0]
    fish_imputation_raw = model.get_imputed_values(normalized=False)[0]
    fish_imputation_theta = model.get_imputed_theta(normalized=False)[0]

    spatial_data_imputed = spatial_data[:, seq_data.var_names]

    spatial_data_imputed.obsm['imputed'] = fish_imputation_norm
    spatial_data_imputed.obsm['imputed_raw'] = fish_imputation_raw
    spatial_data_imputed.obsm['imputed_raw_theta'] =  fish_imputation_theta
    #     del spatial_data_imputed.uns['cell_types']


    spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_seqfish/gimvigat_seqfishdefault1024_32_300_smaple{sample_index}.h5ad")

/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 1 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/model/_utils.py:286: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  warnings.warn(
/gpfs/gibbs/project/zhao/t

Epoch 300/300: 100%|██████████| 300/300 [07:00<00:00,  1.40s/it, loss=50.1, v_num=1]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 300/300: 100%|██████████| 300/300 [07:00<00:00,  1.40s/it, loss=50.1, v_num=1]


/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 1 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend 

...

/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:119: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 1 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend 

Epoch 300/300: 100%|██████████| 300/300 [07:00<00:00,  1.40s/it, loss=50.1, v_num=1]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 300/300: 100%|██████████| 300/300 [07:00<00:00,  1.40s/it, loss=50.1, v_num=1]


OSError: [Errno 122] Driver write request failed (file write failed: time = Fri Nov  3 15:52:40 2023
, filename = '/gpfs/gibbs/pi/zhao/tl688/tangram/data_seqfish/gimvigat_seqfishdefault1024_32_300_smaple9.h5ad', file descriptor = 89, errno = 122, error message = 'Disk quota exceeded', buf = 0x55d388b4f898, total write size = 17704, bytes this sub-write = 17704, bytes actually written = 18446744073709551615, offset = 0)